# Bootstrap Current Self-Consistency

This tutorial demonstrates how to optimize a quasi-symmetric equilibrium to have a self-consistent bootstrap current profile. 
This is performed by minimizing the difference between the toroidal currents $\langle J \cdot B \rangle$ computed from the MHD equilibrium and from the Redl formula. 
The Redl formula is only valid in the limit of perfect quasi-symmetry, so this procedure will not work for other configurations that are not quasi-symmetric. 

There are two methods that can be used, and both will be shown: 

1. Optimize the current profile for self-consistency
2. Iteratively solve the equilibrium with new current profiles

These methods should be equivalent, although one might be faster than the other depending on the particular problem. 

If you have access to a GPU, uncomment the following two lines: 

In [12]:
# from desc import set_device
# set_device("gpu")

In [13]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

In [14]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.size"] = 14

from desc.compat import rescale
from desc.equilibrium import EquilibriaFamily
from desc.examples import get
from desc.grid import LinearGrid
from desc.objectives import (
    BootstrapRedlConsistency,
    FixAtomicNumber,
    FixBoundaryR,
    FixBoundaryZ,
    FixCurrent,
    FixElectronDensity,
    FixElectronTemperature,
    FixIonTemperature,
    FixPsi,
    ForceBalance,
    ObjectiveFunction,
)
from desc.plotting import plot_1d
from desc.profiles import PowerSeriesProfile

As an example, we will reproduce the QA results from [Landreman et al. (2022)](https://doi.org/10.1063/5.0098166). 

We will start with the "precise QA" example equilibrium, scaled to the ARIES-CS reactor size. 

In [15]:
eq0 = get("precise_QA")
eq0 = rescale(eq0, L=("R0", 10), B=("B0", 5.86))

This equilibrium has the vacuum profiles $p = 0 ~\text{Pa}$ and $\frac{2\pi}{\mu_0} I = 0 ~\text{A}$. 
Calculating the bootstrap current requires knowledge of the temperature and density profiles for each species in the plasma. 
We replace the pressure profile with the following kinetic profiles corresponding to $\langle\beta\rangle=2.5\%$: 

$n_e = n_i = 2.38\times10^{20} (1 - \rho^{10}) ~\text{m}^{-3}$

$T_e = T_i = 9.45\times10^{3} (1 - \rho^{2}) ~\text{eV}$

The temperature profiles must be given for both ions and electrons, but only the electron density profile is specified. 
The ion density profile is given by the effective atomic number $Z_{eff}$ as $n_i = n_e / Z_{eff}$. 
The plasma pressure will then be computed as 

$p = e (n_e T_e + n_i T_i)$.

In [16]:
eq0.pressure = None  # must remove the pressure profile before setting kinetic profiles
eq0.atomic_number = PowerSeriesProfile([1])
eq0.electron_density = PowerSeriesProfile(params=[1, -1], modes=[0, 10]) * 2.38e20
eq0.electron_temperature = PowerSeriesProfile(params=[1, -1], modes=[0, 2]) * 9.45e3
eq0.ion_temperature = PowerSeriesProfile(params=[1, -1], modes=[0, 2]) * 9.45e3
# the existing current profile is the vacuum case eq0.current = PowerSeriesProfile([0])

We need to re-solve the equilibrium force balance with the new profiles. 

In [17]:
eq0, _ = eq0.solve(objective="force", optimizer="lsq-exact", verbose=3)

Now we have our initial equilibrium, which does not have a self-consistent bootstrap current: 

In [18]:
fig, ax = plot_1d(eq0, "<J*B> Redl", linecolor="b", lw=2, label="Redl")
fig, ax = plot_1d(eq0, "<J*B>", linecolor="r", lw=2, label="MHD", ax=ax)
ax.legend(loc="best")
ax.set_title("Initial Equilibrium")

We need to create a grid on which to evaluate the boootstrap current self-consistency. 
The bootstrap current is a radial profile, but the grid must have finite poloidal and toroidal resolution to accurately compute flux surface quantities. 
The Redl formula is undefined where the kinetic profiles vanish, so in our example we do not include points at $\rho=0$ or $\rho=1$. 

In [19]:
grid = LinearGrid(
    M=eq0.M_grid,
    N=eq0.N_grid,
    NFP=eq0.NFP,
    sym=eq0.sym,
    rho=np.linspace(1 / eq0.L_grid, 1, eq0.L_grid) - 1 / (2 * eq0.L_grid),
)

Our current profile will be represented as a power series of the form: 

$I = c_0 + c_1 \rho + c_2 \rho^2 + \mathcal{O}(\rho^3)$

Physically, the current should vanish on the magnetic axis so $c_0 = 0$. 
And in order for the MHD equilibrium to be analytic, it should scale as $\mathcal{O}(\rho^2)$ near the magnetic axis so $c_1 = 0$ also. 
However, the Redl bootstrap current formula scales as $\mathcal{O}(\sqrt{\rho})$ near the magnetic axis. 
This is incorrect, because the drift-kinetic equation from the Redl formula does not account for finite orbit width effects that become important near the axis. 

Typically, we use even power series with `sym=True` for all equilibrium profiles to give the desired analycity conditions. 
For bootstrap current optimizations, it is recommended to use the full power series with `sym=False` while also enforcing $c_0 = c_1 = 0$. 
This prevents getting good self-consistency near the magnetic axis, but allows for good agreement throughout the rest of the plasma volume and results in high quality equilibria overall. 


In [20]:
eq0.current = PowerSeriesProfile(np.zeros((eq0.L + 1,)), sym=False)

### 1. Optimization

In this method, we will optimize the current profile to minimize the self-consistency errors evaluated by the `BootstrapRedlConsistency` objective. 
This objective requires the helicity, which for QA is $(M, N) = (1, 0)$. 

In this example we will only optimize the current profile, so all other profiles and the plasma boundary are constrained to be fixed. 
It is recommended to use a very small value for `gtol` when optimizing the bootstrap current. 

In [29]:
eq1 = eq0.copy()

In [30]:
objective = ObjectiveFunction(
    BootstrapRedlConsistency(eq=eq1, grid=grid, helicity=(1, 0)),
)
constraints = (
    FixAtomicNumber(eq=eq1),
    FixBoundaryR(eq=eq1),
    FixBoundaryZ(eq=eq1),
    FixCurrent(eq=eq1, indices=[0, 1]),  # fix c_0=c_1=0 current profile coefficients
    FixElectronDensity(eq=eq1),
    FixElectronTemperature(eq=eq1),
    FixIonTemperature(eq=eq1),
    FixPsi(eq=eq1),
    ForceBalance(eq=eq1),
)
eq1, _ = eq1.optimize(
    objective=objective,
    constraints=constraints,
    optimizer="proximal-lsq-exact",
    maxiter=4,
    gtol=1e-16,  # it is recommended to use a very small gtol
    verbose=3,
)

When plotting the bootstrap current profiles, we see the MHD equilibrium now has very good agreement with the Redl formula. 

In [31]:
fig, ax = plot_1d(eq1, "<J*B> Redl", linecolor="b", lw=2, label="Redl")
fig, ax = plot_1d(eq1, "<J*B>", linecolor="r", lw=2, label="MHD", ax=ax)
ax.legend(loc="best")
ax.set_title("Method 1: Optimization")

### 2. Iterative Solves

In this method, we iteratively solve the equilibrium with updated guesses for the current profile. 
The current profile is computed such that the parallel current is consistent with the Redl formula, according to Equation C3 in [Landreman & Catto (2012)](https://doi.org/10.1063/1.3693187). 
This is the same approach as STELLOPT VBOOT with SFINCS, and it usually converges in only a few iterations. 

In [24]:
eq2 = eq0.copy()
fam2 = EquilibriaFamily(eq2)

In [25]:
niters = 3
for k in range(niters):
    eq2 = eq2.copy()
    # compute new guess for the current profile, consistent with Redl formula
    data = eq2.compute("current Redl", grid)
    current = grid.compress(data["current Redl"])
    rho = grid.compress(data["rho"])
    # fit the current profile to a power series, with c_0=c_1=0
    XX = np.fliplr(np.vander(rho, eq2.L + 1)[:, :-2])
    eq2.c_l = np.pad(np.linalg.lstsq(XX, current, rcond=None)[0], (2, 0))
    # re-solve the equilibrium
    eq2, _ = eq2.solve(objective="force", optimizer="lsq-exact", verbose=3)
    fam2.append(eq2)

We can plot the current profile at each iteration to visualize how it changed: 

In [26]:
fig, ax = plot_1d(fam2[0], "current", linecolor="k", lw=2, label="0")
fig, ax = plot_1d(fam2[1], "current", linecolor="g", lw=2, label="1", ax=ax)
fig, ax = plot_1d(fam2[2], "current", linecolor="b", lw=2, label="2", ax=ax)
fig, ax = plot_1d(fam2[3], "current", linecolor="r", lw=2, label="3", ax=ax)
ax.legend(loc="best")

With this method the MHD equilibrium also has very good agreement with the Redl formula. 

In [27]:
fig, ax = plot_1d(eq2, "<J*B> Redl", linecolor="b", lw=2, label="Redl")
fig, ax = plot_1d(eq2, "<J*B>", linecolor="r", lw=2, label="MHD", ax=ax)
ax.legend(loc="best")
ax.set_title("Method 2: Iterative Solves")

### Comparison

Even though both methods give good self-consistency for the bootstrap current, they do result in slightly different coefficients for the current profile: 

In [32]:
print(eq1.c_l)
print(eq2.c_l)

In this example, the first method of optimization gave better self-consistency but was noticeably slower than the second method of iterative solves. 